In [6]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Load dataset
file_path = "1001.csv"  # Update with the correct path if needed
df = pd.read_csv(file_path)

# Create a directed graph
G = nx.DiGraph()

# Add nodes (companies)
companies = set(df["From Account"]).union(set(df["To Account"]))
for company in companies:
    G.add_node(company)

# Position nodes using spring layout (must be done before using pos)
pos = nx.spring_layout(G, seed=42, k=0.3)

# Separate normal and suspicious transactions
blue_edges = []  # Normal transactions
red_edges = []   # Suspicious transactions
mid_x, mid_y, hover_texts = [], [], []  # Midpoints for suspicious transactions

for _, row in df.iterrows():
    sender = row["From Account"]
    receiver = row["To Account"]
    amount = row["Amount Paid"]

    if row["Suspicious"] == "Yes":
        red_edges.append((sender, receiver))

        # Compute midpoint **after pos is defined**
        x0, y0 = pos[sender]
        x1, y1 = pos[receiver]
        mid_x.append((x0 + x1) / 2)
        mid_y.append((y0 + y1) / 2)
        hover_texts.append(f"From: {sender}<br>To: {receiver}<br>Amount: ${amount:,.2f}")
    else:
        blue_edges.append((sender, receiver))

# Extract node positions
node_x, node_y = zip(*[pos[node] for node in G.nodes()])

# Extract edge positions
blue_edge_x, blue_edge_y = [], []
for edge in blue_edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    blue_edge_x.extend([x0, x1, None])
    blue_edge_y.extend([y0, y1, None])

red_edge_x, red_edge_y = [], []
for edge in red_edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    red_edge_x.extend([x0, x1, None])
    red_edge_y.extend([y0, y1, None])

# Create Plotly traces
blue_edge_trace = go.Scatter(
    x=blue_edge_x, y=blue_edge_y,
    line=dict(width=1, color="blue"),
    hoverinfo="none",  # No hover text for normal transactions
    mode="lines"
)

red_edge_trace = go.Scatter(
    x=red_edge_x, y=red_edge_y,
    line=dict(width=2, color="red"),
    hoverinfo="none",  # No direct hover text on red lines
    mode="lines"
)

# Transparent markers at the midpoints of suspicious transactions for hover text
hover_trace = go.Scatter(
    x=mid_x, y=mid_y,
    mode="markers",
    hoverinfo="text",
    text=hover_texts,
    marker=dict(size=5, color="rgba(255,255,255,0)")  # Invisible markers
)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    text=list(G.nodes()),
    mode="markers+text",
    textposition="top center",
    hoverinfo="text",
    marker=dict(size=10, color="lightblue", line=dict(width=2, color="black"))
)

# Create interactive graph
fig = go.Figure(data=[blue_edge_trace, red_edge_trace, hover_trace, node_trace])
fig.update_layout(
    title="Bank Transaction Network (Hover to See Suspicious Transaction Amounts)",
    showlegend=False,
    hovermode="closest",
    margin=dict(b=0, l=0, r=0, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Show the figure
fig.show()
